In [ ]:
import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
os.environ["WANDB_API_KEY"] = "0"

In [ ]:
from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:
train = pd.read_csv("../input/contradictory-my-dear-watson/train.csv")

In [ ]:
train.head()

In [ ]:
train.premise.values[1]

In [ ]:
train.hypothesis.values[1]

In [ ]:
train.label.values[1]

In [ ]:
labels, frequencies = np.unique(train.language.values,return_counts=True)

plt.figure(figsize = (10,10))
plt.pie(frequencies,labels=labels,autopct='%1.1f%%')
plt.show()

In [ ]:
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
def encode_sentence(s):
   tokens = list(tokenizer.tokenize(s))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)


In [ ]:
encode_sentence("I love machine learning")

In [ ]:
def bert_encode(hypotheses, premises, tokenizer):
    num_examples = len(hypotheses)
  
    sentence1 = tf.ragged.constant([
      encode_sentence(s)
      for s in np.array(hypotheses)])
    sentence2 = tf.ragged.constant([
      encode_sentence(s)
       for s in np.array(premises)])
    cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*sentence1.shape[0]
    input_word_ids = tf.concat([cls, sentence1, sentence2], axis=-1)

    input_mask = tf.ones_like(input_word_ids).to_tensor()

    type_cls = tf.zeros_like(cls)
    type_s1 = tf.zeros_like(sentence1)
    type_s2 = tf.ones_like(sentence2)
    input_type_ids = tf.concat(
      [type_cls, type_s1, type_s2], axis=-1).to_tensor()
    inputs = {
        'input_word_ids': input_word_ids.to_tensor(),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids}

    return inputs


In [ ]:
train_input = bert_encode(train.premise.values, train.hypothesis.values, tokenizer)

In [ ]:
max_len = 259
def build_model():
    bert_encoder = TFBertModel.from_pretrained(model_name)
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32,name="input_word_ids")
    input_mask = tf.keras.Input(shape = (max_len,),dtype=tf.int32,name = "input_mask")
    input_type_ids = tf.keras.Input(shape = (max_len,),dtype=tf.int32,name="input_type_ids")
    
    embedding = bert_encoder([input_word_ids,input_mask,input_type_ids])[0]
    output = tf.keras.layers.Dense(3,activation='softmax')(embedding[:,0,:])
    
    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5),
                 loss = 'sparse_categorical_crossentropy', metrics= ['accuracy'])
    return model

In [ ]:
with strategy.scope():
    model = build_model()
    model.summary()

In [ ]:
model.fit(train_input, train.label.values, epochs = 2, verbose = 1, batch_size=64, validation_split=0.2)

In [ ]:
test = pd.read_csv("../input/contradictory-my-dear-watson/test.csv")
test_input = bert_encode(test.premise.values, test.hypothesis.values, tokenizer)

In [ ]:
test.head()

In [ ]:
test2.tail()

In [ ]:
test2 = pd.DataFrame([['0', '[PAD] '*128, '[PAD] '*128,'en','English']],
                   columns=['id', 'premise', 'hypothesis','lang_abv','language'])

In [ ]:
test3 = test

In [ ]:
test3 = test3.append(test2)

In [ ]:
test_input = bert_encode(test3.premise.values, test3.hypothesis.values, tokenizer)

In [ ]:
predictions = [np.argmax(i) for i in model.predict(test_input)]

In [ ]:
predictions = predictions[:-1]

In [ ]:
len(predictions)

In [ ]:
submission = test.id.copy().to_frame()
submission['prediction'] = predictions

In [ ]:
submission.head()

In [ ]:
submission['id']

In [ ]:
submission.to_csv("submission.csv", index=False)